# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [ ]:
# Next two lines are for dev purposes only 
%load_ext autoreload 
%autoreload 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("Logging works :)")
import pandas as pd
import numpy as np
from pipeline.pipeline_step_1 import run_preprocessing, analyze_new
from pathlib import Path
import glob
import os
from config import settings
COMMON_FILES = Path(settings.common_files)

### STEP 0: Configuration

- Set all necessary variables (videos path and root path) in `pipeline/config.yml`!

#### Override
If you would like to set these values in the noetbook, you can instead 
- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [ ]:
"""NOTE: this is just for overriding the config parameters within notebook"""

# Set the filepath to the genotype folder containing videos
# videos = Path(r"\\mpfi.org\public\sb-lab\DLC_pipeline_Dummy\0_QualityCtrl\Raw_vid_test_new_Code_2\40_BDN1_CsChrim_decap")

# root directory of project (usually overall experiment folder or similar)
# root = Path(r'\\mpfi.org\public\sb-lab\DLC_pipeline_Dummy\0_QualityCtrl') # Root directory of experiment/run, typically genotype or some other overarching directory. 
# Videos must be in dir or a subdir of root 

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
analyze_new()
print("Finished DLC analysis...")

### Step 1b [only for SS files]

In [ ]:
ref_cols = pd.read_csv(COMMON_FILES.joinpath("SS_preprocess_extracols.csv"), header = [0,1,2])
path = Path(settings.videos_path)
for fly_folder in os.listdir(path):
    fly_path = path / fly_folder
    if fly_folder.startswith('N'):
        for cam_file in os.listdir(Path(str(fly_path) + '/SS')):
            
            if cam_file.startswith('H') and cam_file.endswith('_filtered.csv'):
                print(f"Processing file: {cam_file} in {fly_path}")
                camH = pd.read_csv(Path(str(fly_path) + '/SS') / cam_file, header = [0,1,2])
                camH_new = pd.concat([camH, ref_cols], axis = 1)
                
                camH_new.to_csv(Path(str(fly_path) + '/SS') / cam_file, index = False) ## note: replaces the original file 

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [ ]:
run_preprocessing()